# Read input

In [1]:
input00 = open("input00.txt").readlines()

In [2]:
#print(input00)
numTrainingData, numFeatures = (int(s) for s in input00[0].split())

# Create numpy arrays to store training features and labels

In [3]:
import numpy as np
trainingFeatures = np.zeros(shape=(numTrainingData, numFeatures))
trainingLabels = np.zeros(shape=(numTrainingData,), dtype=int)

# Extract features from input dataset

In [4]:
inputFeatures = input00[1:numTrainingData+1]

In [5]:
len(inputFeatures)

4500

# Clean training dataset, remove unnecessary characters and store in arrays

In [6]:
for entryNum in range(trainingFeatures.shape[0]):
    entry = inputFeatures[entryNum].split()
    #print(entry)
    trainingLabels[entryNum] = float(entry[1])
    params = [entry[i] for i in range(2, numFeatures + 2)]
    cleanedParams = [param[param.index(":") + 1:] for param in params]
    trainingFeatures[entryNum] = np.array([float(param) for param in cleanedParams])

In [7]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler()
#trainingFeatures = trainingFeatures[:,[6,7,13]]
trainingFeatures = X_scaler.fit_transform(trainingFeatures)
trainingFeatures.shape

(4500, 23)

# Exract test features

In [8]:
numTestData = int(input00[numTrainingData+1])
print(numTestData)

500


In [9]:
testFeatures = np.zeros(shape=(numTestData, numFeatures))
testNames = []

In [10]:
inputTestFeatures = input00[numTrainingData+2:numTrainingData+2+numTestData]
#print(inputTestFeatures)

In [11]:
len(inputTestFeatures)

500

In [12]:
#trainingLabels = convert_to_onehot(trainingLabels,2)
trainingLabels = np.array([i + 1 if i < 0 else i for i in trainingLabels])


[ 1  1  1 ..., -1 -1  1]


# Clean test dataset, remove unnecessary characters and store in arrays

In [13]:
for entryNum in range(testFeatures.shape[0]):
    entry = inputTestFeatures[entryNum].split()
    testNames.append(entry[0])
    params = [entry[i] for i in range(1, numFeatures + 1)]
    cleanedParams = [param[param.index(":") + 1:] for param in params]
    testFeatures[entryNum] = np.array([float(param) for param in cleanedParams])

In [14]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler()
#testFeatures = testFeatures[:,[6,7,13]] 
testFeatures = X_scaler.fit_transform(testFeatures)
testFeatures.shape

(500, 23)

In [15]:
def convert_to_onehot(labels, num_classes):
    labels = labels.reshape(-1)
    return np.eye(num_classes, dtype=int)[labels]

In [16]:
def batch(features, labels, idx, batch_size):
    idx = idx % (features.shape[0] // batch_size)
    if (idx+1) * batch_size < labels.shape[0]:
        return features[idx * batch_size:(idx+1) * batch_size], labels[idx * batch_size:(idx+1) * batch_size]
    else:
        return features[idx * batch_size:], labels[idx * batch_size:]

# Read labels for test dataset

In [17]:
content = open("output00.txt").readlines()
content = [line.strip('\n') for line in content]
cleanedContent = [entry[entry.index(" ") + 1:] for entry in content]

testLabels = np.array([float(s) for s in cleanedContent], dtype=int)
testLabels = np.array([i + 1 if i < 0 else i for i in testLabels])


In [18]:
trainingFeatures[:,1].T

array([ 2.44352513,  1.19620926,  0.16102307, ..., -1.1921951 ,
       -0.18189074,  0.27827511])

In [19]:
type(trainingFeatures[0,0])

numpy.float64

In [20]:
testLabels.shape

(500,)

# Build Neural Network

In [21]:
import tensorflow as tf

learning_rate = 0.001

num_labels = 2

hidden_nodes_layer1 = 128

hidden_nodes_layer2 = 64

hidden_nodes_layer3 = 32

dropout_keep_prob = tf.placeholder(tf.float64)

X = tf.placeholder(tf.float64, [None, trainingFeatures.shape[1]])
y_ = tf.placeholder(tf.int32, [None])

W_hidden1 = tf.Variable(tf.random_normal([trainingFeatures.shape[1], hidden_nodes_layer1], dtype=tf.float64))
b_hidden1 = tf.Variable(tf.random_normal([hidden_nodes_layer1], dtype=tf.float64))

hidden1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden1), b_hidden1))


W_hidden2 = tf.Variable(tf.random_normal([hidden_nodes_layer1, hidden_nodes_layer2], dtype=tf.float64))
b_hidden2 = tf.Variable(tf.random_normal([hidden_nodes_layer2], dtype=tf.float64))

hidden2 = tf.nn.relu(tf.add(tf.matmul(hidden1, W_hidden2), b_hidden2))

W_hidden3 = tf.Variable(tf.random_normal([hidden_nodes_layer2, hidden_nodes_layer3], dtype=tf.float64))
b_hidden3 = tf.Variable(tf.random_normal([hidden_nodes_layer3], dtype=tf.float64))

hidden3 = tf.nn.relu(tf.add(tf.matmul(hidden2, W_hidden3), b_hidden3))

dropout = tf.nn.dropout(hidden3, keep_prob=dropout_keep_prob)

W_out = tf.Variable(tf.random_normal([hidden_nodes_layer3, num_labels], dtype=tf.float64))
b_out = tf.Variable(tf.random_normal([num_labels], dtype=tf.float64))

y = tf.add(tf.matmul(dropout, W_out), b_out)


cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=y_))
tf.summary.scalar('cross_entropy', cross_entropy)
merged = tf.summary.merge_all()
train_step = tf.train.AdamOptimizer(learning_rate, epsilon=1e-8, beta1=0.9, beta2=0.999).minimize(cross_entropy)

In [ ]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

saver = tf.train.Saver()

#saver.restore(sess, tf.train.latest_checkpoint('checkpoint/'))

log_dir = './logs'

train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
test_writer = tf.summary.FileWriter(log_dir + '/test', sess.graph)

for i in range(10000000):
    _, loss, train_summary = sess.run([train_step, cross_entropy, merged], feed_dict={X: trainingFeatures, y_:trainingLabels, dropout_keep_prob:0.3})
    train_writer.add_summary(train_summary, i)
    if i%100 == 0:
        saver.save(sess, save_path='checkpoint/lin_classifier', global_step=i)
        print('step {}:{}'.format(i, loss))
    if i%200 == 0:
        _, test_loss, test_summary = sess.run([train_step, cross_entropy, merged], feed_dict={X: testFeatures, y_:testLabels, dropout_keep_prob:1.0})
        test_writer.add_summary(test_summary, i)
        print('\ntest loss:', test_loss)

step 0:719.6211680596717

test loss: 518.902573509
step 100:94.39944299153422
step 200:15.298795104254777

test loss: 14.0296561553
step 300:3.4066979790526246
step 400:1.4611944120457896

test loss: 3.51291204139
step 500:0.8004473880195582
step 600:0.8626167478859974

test loss: 2.73947248735
step 700:0.6527927013386755
step 800:0.8201253957887744

test loss: 2.49268184612
step 900:0.7785411256723811
step 1000:0.6286360566325049

test loss: 2.40280654009
step 1100:0.6292043415959748
step 1200:0.6147869485996895

test loss: 2.3361854444
step 1300:0.6186000125572491
step 1400:0.6175159877512071

test loss: 2.20746349211
step 1500:0.6188202059823814
step 1600:0.614920826850668

test loss: 2.0852182151
step 1700:0.6162225367327472
step 1800:0.6111147142426

test loss: 1.94994138755
step 1900:0.611985153922199
step 2000:0.5999186858791112

test loss: 1.79677901167
step 2100:0.5943383950560301
step 2200:0.5947048257662809

test loss: 1.64716836807
step 2300:0.5894139239065762
step 2400:0.5

step 19100:0.18810997518684333
step 19200:0.18418867453542134

test loss: 0.49413294192
step 19300:0.18898592566324174
step 19400:0.18394207054643918

test loss: 0.459178172539
step 19500:0.18069908828372058
step 19600:0.18955971253477594

test loss: 0.448245445659
step 19700:0.18662268089311881
step 19800:0.1838757477246177

test loss: 0.43415642432
step 19900:0.17677190413103688
step 20000:0.18336482052830333

test loss: 0.431309829129
step 20100:0.1772354996444064
step 20200:0.17816607624724506

test loss: 0.436954544077
step 20300:0.18258894614957574
step 20400:0.18117400117744445

test loss: 0.417492739186
step 20500:0.18712191020690344
step 20600:0.18347649701364668

test loss: 0.41987094453
step 20700:0.18317328122753995
step 20800:0.18270289905504145

test loss: 0.385260343892
step 20900:0.17823521789044555
step 21000:0.17932563655690173

test loss: 0.388760582882
step 21100:0.17967010783157147
step 21200:0.17762747161942843

test loss: 0.374886642212
step 21300:0.1861193687641

step 37700:0.13294293950162314
step 37800:0.13348740689042946

test loss: 0.112885927097
step 37900:0.12798504961639218
step 38000:0.13124831041130242

test loss: 0.14322003864
step 38100:0.13300106148792804
step 38200:0.13232775579768788

test loss: 0.123619770887
step 38300:0.1307816875416424
step 38400:0.13177268449968113

test loss: 0.103131399939
step 38500:0.12638583762080777
step 38600:0.12577753442399764

test loss: 0.110003927632
step 38700:0.1300913201361569
step 38800:0.1274305785670005

test loss: 0.106283638737
step 38900:0.12089251798970084
step 39000:0.1271761969246486

test loss: 0.113294899987
step 39100:0.13175612425346375
step 39200:0.1287938637331849

test loss: 0.121365075171
step 39300:0.12515082303245437
step 39400:0.12467654329487363

test loss: 0.101236925947
step 39500:0.12881086265697403
step 39600:0.1254243144626434

test loss: 0.135277375859
step 39700:0.123838693936212
step 39800:0.1229222266416718

test loss: 0.097664670215
step 39900:0.1259389815370136
s

step 56200:0.09890948973487707

test loss: 0.092258557482
step 56300:0.09749883091461996
step 56400:0.10075027942634966

test loss: 0.0868314555163
step 56500:0.0971803248400361
step 56600:0.09582115828124413

test loss: 0.158323990003
step 56700:0.10136459843393783
step 56800:0.09790443373013583

test loss: 0.0855492574299
step 56900:0.09953980171692264
step 57000:0.09704331064035866

test loss: 0.0919386628769
step 57100:0.09699798480794888
step 57200:0.09885399680150665

test loss: 0.0945650572036
step 57300:0.09930581732081063
step 57400:0.09726103690489012

test loss: 0.079585536753
step 57500:0.09728608462310535
step 57600:0.09534102345151657

test loss: 0.0819548997313
step 57700:0.09635274453934001
step 57800:0.09943431724119607

test loss: 0.0790821162157
step 57900:0.09389327562609773
step 58000:0.0993051557898135

test loss: 0.083303884995
step 58100:0.09679442044322661
step 58200:0.09856291921439318

test loss: 0.0774842105297
step 58300:0.09972792111440366
step 58400:0.096

step 74500:0.07783802063346633
step 74600:0.07791482443825754

test loss: 0.0760329038644
step 74700:0.08035781645568446
step 74800:0.07830864175612551

test loss: 0.0623673560109
step 74900:0.07774510686690998
step 75000:0.07949103983784155

test loss: 0.0834902004386
step 75100:0.08138161185825238
step 75200:0.07605820064371713

test loss: 0.123226199407
step 75300:0.08018743474773647
step 75400:0.07792082446897303

test loss: 0.0793072601706
step 75500:0.07672443976566597
step 75600:0.07720689682303916

test loss: 0.0591985687193
step 75700:0.07397064229434717
step 75800:0.07536465087262553

test loss: 0.0621714247216
step 75900:0.07652559956495911
step 76000:0.07628918574958314

test loss: 0.0628832150232
step 76100:0.076136774152928
step 76200:0.07391100607411474

test loss: 0.104925310537
step 76300:0.0806892281913222
step 76400:0.07593734078348348

test loss: 0.115173163153
step 76500:0.07895618215932357
step 76600:0.0805381828600303

test loss: 0.231781265553
step 76700:0.07708